In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC
from time import time  # measure running time
from sklearn.neural_network import MLPClassifier  # neural network
from sklearn.model_selection import train_test_split  # split train test size
from sklearn.naive_bayes import BernoulliNB, GaussianNB  # naive bayesian
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix  # metrics
from sklearn.ensemble import RandomForestClassifier, VotingClassifier  # ensemble methods

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
df = pd.read_csv('../input/european-cup2/International football results from 1872 to 2021.csv')
df.head()

In [ ]:
df_FIFA_all = df[df['tournament'].str.contains('FIFA', regex=True)]

df_FIFA = df_FIFA_all[df_FIFA_all['tournament']=='FIFA World Cup']

df_FIFA.head()

In [ ]:
df_FIFA.loc[:,'date'] = pd.to_datetime(df_FIFA.loc[:,'date'])

df_FIFA['year'] = df_FIFA['date'].dt.year

df_FIFA['diff_score'] = df_FIFA['home_score']-df_FIFA['away_score']

df_FIFA['win_team'] = ''

df_FIFA['diff_score'] = pd.to_numeric(df_FIFA['diff_score'])

In [ ]:
df_FIFA.loc[df_FIFA['diff_score']> 0, 'win_team'] = df_FIFA.loc[df_FIFA['diff_score']> 0, 'home_team']
df_FIFA.loc[df_FIFA['diff_score']< 0, 'win_team'] = df_FIFA.loc[df_FIFA['diff_score']< 0, 'away_team']
df_FIFA.loc[df_FIFA['diff_score']== 0, 'win_team'] = 'Draw'
df_FIFA.head()
# The second method to get the winners
def find_win_team(df):
   winners = []
   for i, row in df.iterrows():
       if row['home_score'] > row['away_score']:
           winners.append(row['home_team'])
       elif row['home_score'] < row['away_score']:
           winners.append(row['away_team'])
       else:
           winners.append('Draw')
   return winners
df_FIFA['winner'] = find_win_team(df_FIFA)
df_FIFA.head()

In [ ]:

s = df_FIFA.groupby('win_team')['win_team'].count()

s.sort_values(ascending=False, inplace=True)

s.drop(labels=['Draw'], inplace=True)

In [ ]:
s.head(20).plot(kind='bar', figsize=(10,6), title='Top 20 Winners of Euro Cup')

In [ ]:
df_score_home = df_FIFA[['home_team', 'home_score']]

column_update = ['team', 'score']

df_score_home.columns = column_update

df_score_away = df_FIFA[['away_team', 'away_score']]

df_score_away.columns = column_update

df_score = pd.concat([df_score_home,df_score_away], ignore_index=True)

s_score = df_score.groupby('team')['score'].sum()

s_score.sort_values(ascending=False, inplace=True)

s_score.sort_values(ascending=True, inplace=True)

s_score.tail(20).plot(kind='barh', figsize=(10,6), title='Top 20 in Total Scores of World Cup')

In [ ]:
team_list = ['Russia', 'Germany', 'Portugal',  'Belgium', 'Poland', 'France',

            'Spain', 'Switzerland', 'England','Uruguay', 'Croatia',

           'Denmark', 'Iceland', 'Costa Rica', 'Sweden', 'Senegal', "Italy"
]
for item in team_list:
   if item not in s_score.index:
       print(item)

In [ ]:
df_top32 = df_FIFA[(df_FIFA['home_team'].isin(team_list))&(df_FIFA['away_team'].isin(team_list))]
s_32 = df_top32.groupby('win_team')['win_team'].count()
s_32.sort_values(ascending=False, inplace=True)
s_32.drop(labels=['Draw'], inplace=True)
s_32.sort_values(ascending=True,inplace=True)
s_32.plot(kind='barh', figsize=(8,12), title='Top 32 of World Cup since year 1872')

df_score_home_32 = df_top32[['home_team', 'home_score']]
column_update = ['team', 'score']
df_score_home_32.columns = column_update
df_score_away_32 = df_top32[['away_team', 'away_score']]
df_score_away_32.columns = column_update
df_score_32 = pd.concat([df_score_home_32,df_score_away_32], ignore_index=True)
s_score_32 = df_score_32.groupby('team')['score'].sum()
s_score_32.sort_values(ascending=False, inplace=True)
s_score_32.sort_values(ascending=True, inplace=True)
s_score_32.plot(kind='barh', figsize=(8,12), title='Top 32 in Total Scores of World Cup since year 1872')